**Implementación de Modelo Regresion**

Objetivo: Predecir la calificación final o promedio.

Integrantes:

- Angel
- Diego 
- Felipe 
- Jair Salvador 
- José 
- Yahir

In [ ]:
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
#from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import statsmodels.api as sm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
#Cargar el DataSet
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Proyecto Final/StudentsPerformance.csv')



In [ ]:
#Ver el Dataset
df.head()

In [ ]:
# Crear variable objetivo: calificación final (promedio)
df['perfil_fortaleza'] = df['math score'] - ((df['reading score'] + df['writing score']) / 2)


El perfil de fortaleza se obtiene comparando el puntaje de matemáticas con el promedio de lectura y escritura.

Su objetivo es identificar si un estudiante muestra mayor fortaleza en matemáticas o en habilidades verbales.

* Un valor positivo indica que el puntaje de matemáticas es mayor que el promedio verbal, por lo que el estudiante tiene un mejor rendimiento en matemáticas.
* Un valor negativo significa que el promedio de lectura y escritura es mayor que el de matemáticas, lo que refleja una fortaleza en habilidades verbales.
* Si el valor es cercano a cero, el desempeño del estudiante es equilibrado entre ambas áreas.

Por ejemplo:
* Un valor de +10 señala que el estudiante obtiene 10 puntos más en matemáticas que en su promedio verbal.
* Un valor de –8 muestra que el estudiante tiene un rendimiento 8 puntos mejor en lectura/escritura que en matemáticas.
* Un valor de 0 representa un desempeño balanceado.

Ejemplos:
- Estudiante 0: 72 – (72 + 74)/2 = –1 (ligera inclinación verbal)
- Estudiante 1: 69 – (90 + 88)/2 = –20 (fuerte inclinación verbal)
- Estudiante 2: 90 – (95 + 93)/2 = –4 (relativamente balanceado)


Esto nos muestra una marcada fortaleza en alguna de estas áreas (ya sea matemáticas o verbal), esta diferencia puede relacionarse con su rendimiento global.

Por ejemplo, un alumno consistentemente fuerte en ambas áreas tenderá a tener un promedio final más alto, mientras que una diferencia muy negativa o muy positiva puede indicar desequilibrios que también influyen en su desempeño global.

In [ ]:
# Ver al dataset con la nueva variable
df.head()

**PREPARAR DATOS PARA EL MODELO FINAL**

In [ ]:
# Usamos todas las variables para predecir promedio final
X = df.drop(['perfil_fortaleza', 'math score', 'reading score', 'writing score'], axis=1)
y = df['perfil_fortaleza']


- X: Variables predictoras (caracteristicas) - eliminamos las columnas que no usaremos
- y: Variable objetivo (objetivo) - el promedio final que queremos predecir

In [ ]:
#Variables predictoras después de eliminar las puntuaciones individuales
print(X.columns.tolist())


In [ ]:
# Convertir variables categóricas a variables dummy ( Usar drop_first=True para evitar multicolinealidad)
X_encoded = pd.get_dummies(X, drop_first=True)

print(f"Variables predictoras: {X_encoded.shape[1]}")
print(f"Tamaño del dataset: {X_encoded.shape[0]}")

Convertimos variables categóricas como "género", "grupo étnico" y "nivel educativo de padres" en formato numérico (0 y 1) para que el modelo pueda entenderlas.

Resultado: 12 variables predictoras listas para el modelo.

**DIVIDIR DATOS EN ENTRENAMIENTO Y PRUEBA**

Dividimos los datos en:
- 80% para entrenar el modelo (X_train, y_train)
- 20% para evaluar el modelo (X_test, y_test)

random_state=42 asegura que la división sea reproducible

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


**SELECCIÓN DE VARIABLES (ELIMINACIÓN HACIA ATRÁS)**

In [ ]:
X_selected = X_train.copy()

# Agregar constante
X = sm.add_constant(X, has_constant='add')

#X_train_const = sm.add_constant(X_selected)

# Umbral de significancia (típicamente 0.05)
significance_level = 0.05

# Proceso iterativo de eliminación hacia atrás
while True:
    # Se ajusta un modelo de regresión lineal OLS (Ordinary Least Squares)
    #Busca una relación lineal entre una variable dependiente y y una o más variables independientes X.
    # Minimiza el error cuadrático total
    model = sm.OLS(y_train, X_selected).fit()
    p_values = model.pvalues
    max_pval = p_values.max()
    if max_pval > significance_level:
        excluded_variable = p_values.idxmax()
        print(f"Eliminando: {excluded_variable} (p={max_pval:.4f})")
        X_selected = X_selected.drop(columns=[excluded_variable])
    else:
        break

# Resultado final
print("\nResumen del modelo final:")
print(model.summary())


# Nota: OLS (Ordinary Least Squares) es el método de mínimos cuadrados para regresión
# Iterativamente eliminamos la variable con mayor p-value hasta que todas
# sean significativas (p < 0.05)


Objetivo: Quitar variables no significativas (p-value > 0.05)

* Todas las variables son significativas
* Ninguna variable eliminada

**Variables más importantes**

gender_male: +11.17 > Muy significativa
race/ethnicity_group C: -8.86 > Muy significativa  
lunch_standard: +1.76 > Significativa

# **ANÁLISIS DE RESIDUALES**

In [ ]:
residual = model.resid
fitted = model.fittedvalues

In [ ]:
residual.mean()

Residual medio = -0.67: El modelo tiende a predecir valores ligeramente más altos que los reales

* Distribución normal: Los errores son aleatorios, no hay patrones sistemáticos
* Buena calibración: El modelo está bien ajustado según nosotros.

In [ ]:
sns.histplot(residual, kde=True)
plt.title('Distribución de los Residuales')
plt.xlabel('Residuales {y_real - y_predicho}')
plt.ylabel('Frecuencia')
plt.show()

* Forma de campana → Distribución normal
* Centrada en -0.67 → Ligero sesgo de sobreestimación
* Colas delgadas → Pocos errores extremos

**ENTRENAR MODELO DE REGRESIÓN LINEAL**

In [ ]:
X_train_selected = X_train[X_selected.columns]
modelo_final = LinearRegression()
modelo_final.fit(X_train_selected, y_train)


**HACER PREDICCIONES**

In [ ]:
X_test_selected = X_test[X_selected.columns]
y_pred = modelo_final.predict(X_test_selected)


**Grafica de predicciones vs valores reales**

In [ ]:
# Gráfico de predicciones vs. valores reales
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title("Valores Reales vs. Predicciones")

min_val = min(min(y_test), min(y_pred))  # Encuentra el valor mínimo entre y_test y y_pred
max_val = max(max(y_test), max(y_pred))  # Encuentra el valor máximo entre y_test y y_pred
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--')  # Dibuja la línea de identidad

plt.show()

* Puntos cerca de línea roja: Hay buenas predicciones
* Dispersión moderada: Algunos errores esperados
* Relación lineal clara: El modelo captura la tendencia general

Patrones identificados:
* Rango central (-10 a +10): máxima precisión
* Valores extremos: Hay mayor error de predicción
* Tendencia lineal: El modelo es apropiado para estos datos.

In [ ]:
print("Ejemplos de predicciones vs valores reales:")
comparacion = pd.DataFrame({
    'Real': y_test.values[:10],
    'Predicho': y_pred[:10],
    'Error': np.abs(y_test.values[:10] - y_pred[:10])
})
print(comparacion.round(2))

Para valores cercanos a cero (habilidades balanceadas), el modelo es muy preciso.
* Real: 3.5, Predicho: 3.67, Error: 0.17
* Real: -7.5, Predicho: -9.25, Error: 1.75

El modelo captura la dirección pero subestima la magnitud
* Real: -16.5, Predicho: -12.23, Error: 4.27
* Real: 7.5, Predicho: 4.80, Error: 2.70

Perfiles extremos son difíciles de predecir con las variables actuales
* Real: 6.0, Predicho: -7.77, Error: 13.77
* Real: 14.5, Predicho: 1.68, Error: 12.82


**EVALUACIÓN DEL MODELO - MÉTRICAS REQUERIDAS**

In [ ]:
#Evaluar el modelo usando el Error Cuadrático Medio (MsE)
mse = mean_squared_error(y_test, y_pred)
print(f"Error Cuadrático Medio (MSE): {mse}")

MSE (Error Cuadrático Medio): 29.09 valor aceptable para el rango de la variable.

In [ ]:
#RMSE (Root Mean Squared Error)
rmse = np.sqrt(mse)
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse}")

Cuando hay errores grandes, estos son de hasta 5.39 puntos

In [ ]:
# MAE (Mean Asbsolut Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Error Absoluto Medio (MAE): {mae}")

MAE = 4.15: El error promedio es de 4.22 puntos en la escala del perfil de fortaleza

CONCLUSIONES FINALES

**Fortalezas del modelo:**
* Variables significativas identificadas correctamente
* Residuales bien comportados (distribución normal)
* Precisión aceptable para análisis grupales

El modelo predice con 4.22 puntos de precisión el perfil académico de los estudiantes, lo que permite identificar aquellos con mayor potencial de éxito académico basado en su fortaleza en matemáticas versus habilidades verbales, lo que nos permite predecir el promedio final o promedio.

In [ ]:
!apt-get install -y texlive-xetex texlive-fonts-recommended texlive-latex-recommended texlive-latex-extra pandoc
!jupyter nbconvert --to pdf '/content/drive/MyDrive/Colab Notebooks/Proyecto Final/ImplementacionModeloRegresion.ipynb'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
pandoc set to manually installed.
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono
  fonts-texgyre fonts-urw-base35 libapache-pom-java libcommons-logging-java
  libcommons-parent-java libfontbox-java libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1 libruby3.0
  libsynctex2 libteckit0 libtexlua53 libtexluajit2 libwoff1 libzzip-0-13
  lmodern poppler-data preview-latex-style rake ruby ruby-net-telnet
  ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0 rubygems-integration t1utils
  teckit tex-common tex-gyre texlive-base texlive-binaries texlive-latex-base
  texlive-pictures texlive-plain-generic tipa xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf libavalon-frame